# Creating an Einstein Toolkit thorn:
## The scalar wave equation

The goal of this tutorial is to construct an Einstein Toolkit (ETK) thorn that will evolve some initial data according to the scalar wave equation, subject to appropriate boundary conditions. This thorn should function identically to the $\text{WaveToy}$ and $\text{IDScalarWaveC}$ thorns included in the ETK.

### Evolving the Scalar Wave equation

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. The scalar wave right-hand sides and plane wave initial data are already built by [$\text{Tutorial-ScalarWave.ipynb}$](Tutorial-ScalarWave.ipynb), so we can simply import them to use here. We will also need to instruct the system to create the directories for our thorn.

We will begin the the evolution equations.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1b: Call the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module,
#         which should do exactly the same as in Steps 1-10 above.
import ScalarWave.ScalarWave_RHSs as swrhs
swrhs.ScalarWave_RHSs()

!mkdir ScalarWave     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir ScalarWave/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.


Now, we need to output these expressions to C code. We will do this the same way as in the Weyl scalars ETK tutorial: export these to a .h file, which can then be included in a main .c file to do the calculations.

In [2]:
# First, we will write the right-hand sides.
scalar_RHSs_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=swrhs.uu_rhs),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=swrhs.vv_rhs),]
scalar_RHSs_CcodeKernel = fin.FD_outputC("returnstring",scalar_RHSs_to_print)

scalar_RHSs_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",scalar_RHSs_CcodeKernel)

with open("ScalarWave/src/ScalarWave_RHSs.h", "w") as file:
    file.write(str(scalar_RHSs_looped))


We will need to write C code to call these header files we have output. To do this, we can follow the examples of WaveToyC and our Weyl Scalar ETK thorn.

In [ ]:
%%writefile ScalarWave/src/ScalarWave.c
#include <stddef.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void ScalarWave_evolution(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
#include "ScalarWave_RHSs.h" 
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void ScalarWave_Boundaries(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  if (bctype && (Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "scalarwave::uuGF", bctype) < 0 || 
                 Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "scalarwave::vvGF", bctype) < 0))
  {
    CCTK_WARN (0, "ScalarWave_Boundaries: Error selecting boundary condition");
  }
}


We will start with the file $\text{interface.ccl}$. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the scalars $\text{uuGF}$ and $\text{vvGF}$ to be visible to other thorns by using the keyword "public". 

In [ ]:
%%writefile ScalarWave/interface.ccl
implements ScalarWave

inherits: grid

USES INCLUDE: Symmetry.h

public:
cctk_real scalar_fields type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  uuGF,vvGF
} "The evolved scalar fields"


We will now write the file $\text{param.ccl}$. This file allows the listed parameters to be set at runtime. We also give allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). We first set a Keyword to determine our boundary conditions; this is not implemented yet, but more options can be added later. We also set the number of timelevels we will store in memory.

In [ ]:
%%writefile ScalarWave/param.ccl
restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "none"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3


Finally, we will write the file $\text{schedule.ccl}$; its official documentation is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). This file dictates when the various parts of the thorn will be run. We first assign storage for both scalar gridfunctions, and then specify that we want our code run 

In [ ]:
%%writefile ScalarWave/schedule.ccl
STORAGE: scalar_fields[timelevels]

We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile so far.

In [ ]:
%%writefile ScalarWave/src/make.code.defn
SRCS = ScalarWave.c

### Scalar Wave Initial Data

Now, we will move on to a second thorn. This thorn will set the initial data.

We will now output the C code for the header file to set plane wave initial data that will be evolved by the first thorn we made in this tutorial.

In [ ]:
# Step 1c: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid
swid.InitialData_PlaneWave()

!mkdir ScalarWave     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir ScalarWave/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Second, we will write the initial data.
scalar_PWID_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=swid.uu_ID),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=swid.vv_ID),]
scalar_PWID_CcodeKernel = fin.FD_outputC("returnstring",scalar_PWID_to_print)

scalar_PWID_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",scalar_PWID_CcodeKernel)

with open("ScalarWave/src/ScalarWave_PWID.h", "w") as file:
    file.write(str(scalar_PWID_looped))
